In [ ]:
import requests
import base64
import time
import pandas as pd
import re
import random
from datetime import datetime

# ==========================================
# AYARLAR
# ==========================================
TOKEN = "" 

headers = {}

# Hedeflenen toplam repo sayısı
TARGET_COUNT = 7000 

# Maksimum dosya sayısı limiti (Bunu aşarsa repo pas geçilir)
MAX_FILE_LIMIT = 100

VALID_LICENSES = [
    "mit", "apache-2.0", "bsd-2-clause",
    "bsd-3-clause", "cc-by-4.0", "cc0-1.0"
]

data = []
count = 0

# ==========================================
# YARDIMCI FONKSİYONLAR
# ==========================================

def clean_excel_string(s):
    """Excel'in kabul etmeyeceği karakterleri temizler."""
    if s is None: return ""
    # Excel hücre limiti yaklaşık 32k karakterdir, taşmayı önlemek için kırpma yapılabilir.
    # Şimdilik sadece illegal karakterleri temizliyoruz.
    s = re.sub(r"[\x00-\x08\x0B\x0C\x0E-\x1F]", "", s)
    return s

def safe_get(url):
    """
    Hata aldığında akıllıca bekleyen istek fonksiyonu.
    Rate Limit (403) hatası alırsa, limitin sıfırlanacağı zamana kadar uyur.
    """
    while True:
        try:
            r = requests.get(url, headers=headers, timeout=20)
            
            # 1. BAŞARILI DURUM
            if r.status_code == 200:
                return r.json()
            
            # 2. RATE LIMIT (HIZ SINIRI) KONTROLÜ (403 veya 429)
            if r.status_code == 403 or r.status_code == 429:
                remaining = int(r.headers.get("x-ratelimit-remaining", 1))
                reset_time = int(r.headers.get("x-ratelimit-reset", 0))
                
                if remaining == 0:
                    current_ts = int(time.time())
                    sleep_seconds = reset_time - current_ts + 10 
                    
                    if sleep_seconds > 0:
                        reset_date = datetime.fromtimestamp(reset_time).strftime('%H:%M:%S')
                        print(f"\n🛑 API KOTASI DOLDU! Saat {reset_date} zamanına kadar bekleniyor ({sleep_seconds/60:.1f} dakika)...")
                        time.sleep(sleep_seconds)
                        print("♻️ UYANILDI! Kota yenilendi, işleme devam ediliyor...")
                        continue 
                
                print(f"⚠️ Çok hızlı istek algılandı. 2 dakika soğutuluyor...")
                time.sleep(120)
                continue

            # 3. BULUNAMADI (404)
            if r.status_code == 404:
                return None
            
            # 4. DİĞER HATALAR
            print(f"⚠️ Beklenmedik HTTP Hatası: {r.status_code} | URL: {url}")
            return None

        except requests.exceptions.ConnectionError:
            print("⚠️ İnternet bağlantısı koptu, 30 saniye bekleniyor...")
            time.sleep(30)
        except Exception as e:
            print(f"⚠️ Genel Hata: {e}")
            return None

def get_file_content(owner, repo, path):
    """Dosya içeriğini çeker ve Base64'ten decode eder."""
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{path}"
    js = safe_get(url)
    
    if js and isinstance(js, dict) and js.get("encoding") == "base64":
        try:
            return base64.b64decode(js["content"]).decode(errors="ignore")
        except:
            return None
    return None

def get_commit_date(owner, repo):
    """Reponun son commit tarihini alır."""
    url = f"https://api.github.com/repos/{owner}/{repo}/commits?per_page=1"
    js = safe_get(url)
    if js and isinstance(js, list) and len(js) > 0:
        return js[0]["commit"]["author"]["date"]
    return ""

def is_valid_license(repo):
    """Lisans kontrolü yapar."""
    lic = repo.get("license")
    return lic and lic.get("key") and lic["key"].lower() in VALID_LICENSES

# ==========================================
# ANA PROGRAM DÖNGÜSÜ
# ==========================================

print("🚀 Tarama Başlıyor... (Dosya sayısı > 100 olan repolar atlanacak)")

# Yıl döngüsü
for year in range(2010, 2021):
    for month in range(1, 13):

        if count >= TARGET_COUNT:
            break

        start = f"{year}-{month:02d}-01"
        if month == 12:
            end = f"{year + 1}-01-01"
        else:
            end = f"{year}-{month+1:02d}-01"

        print(f"\n📆 TARANIYOR: {year}-{month:02d}")

        page = 1
        while True: 
            if count >= TARGET_COUNT:
                break

            search_query = (
                f"language:python "
                f"pushed:{start}..{end} "
                "stars:>0 "
                "license:mit license:apache-2.0 license:bsd-2-clause "
                "license:bsd-3-clause license:cc-by-4.0 license:cc0-1.0"
            )

            url = f"https://api.github.com/search/repositories?q={search_query}&per_page=100&page={page}"
            js = safe_get(url)

            if not js or "items" not in js or len(js["items"]) == 0:
                break 

            for repo in js["items"]:
                if count >= TARGET_COUNT:
                    break

                if not is_valid_license(repo):
                    continue

                owner = repo["owner"]["login"]
                repo_name = repo["name"]
                repo_html_url = repo["html_url"]
                
                # Repo dosya ağacını çek
                tree_url = f"https://api.github.com/repos/{owner}/{repo_name}/git/trees/HEAD?recursive=1"
                tree = safe_get(tree_url)
                
                if not tree or "tree" not in tree:
                    continue

                # Sadece .py dosyalarını bul
                py_files = [f for f in tree["tree"] if f["path"].endswith(".py")]
                
                # --- YENİ EKLENEN KONTROL ---
                # Eğer dosya sayısı 0 ise veya MAX_FILE_LIMIT (100)'den fazlaysa atla
                if not py_files:
                    continue
                
                if len(py_files) > MAX_FILE_LIMIT:
                    # Bilgi vermek için ekrana yazdırıp pas geçiyoruz
                    # print(f"   ⏭️ {repo_name} atlanıyor (Dosya sayısı: {len(py_files)} > {MAX_FILE_LIMIT})")
                    continue
                # -----------------------------

                full_code_content = ""
                
                print(f"   [{count+1}/{TARGET_COUNT}] {repo_name} indiriliyor... ({len(py_files)} dosya)")

                for f in py_files:
                    content = get_file_content(owner, repo_name, f["path"])
                    if content:
                        clean_content = clean_excel_string(content)
                        full_code_content +=clean_content
                    
                # Kod içeriği anlamlı boyuttaysa kaydet
                if len(full_code_content.strip()) > 50:
                    commit_date = get_commit_date(owner, repo_name)
                    
                    data.append({
                        "Kullanıcı": owner,
                        "Repo": repo_html_url,
                        "Tarih": commit_date,
                        "Kod": full_code_content,
                        "Person 1": "Evet"
                    })
                    
                    count += 1
                
                if count % 100 == 0:
                    print("💾 Ara kayıt yapılıyor...")
                    df_temp = pd.DataFrame(data)
                    df_temp.to_excel(f"github_backup_{count}.xlsx", index=False)

            page += 1
            time.sleep(2)

    if count >= TARGET_COUNT:
        print("\n🏁 Hedeflenen sayıya ulaşıldı!")
        break

# ==========================================
# KAYIT İŞLEMİ
# ==========================================
print("\n💾 Tüm veriler Excel'e kaydediliyor...")
df = pd.DataFrame(data)
df["Kod"] = df["Kod"].apply(clean_excel_string)

output_filename = f"github_python_filtered_{count}_repos.xlsx"
df.to_excel(output_filename, index=False)

print(f"✅ İŞLEM TAMAMLANDI! Dosya adı: {output_filename}")

🚀 Tarama Başlıyor... (Dosya sayısı > 100 olan repolar atlanacak)

📆 TARANIYOR: 2010-01
   [1/7000] google_closure_compiler indiriliyor... (1 dosya)
   [2/7000] django-admin-histograms indiriliyor... (7 dosya)
   [3/7000] awesomestream indiriliyor... (13 dosya)
   [4/7000] newf indiriliyor... (2 dosya)
   [5/7000] django-social-bookmarking indiriliyor... (17 dosya)
   [6/7000] DrEval indiriliyor... (8 dosya)
   [7/7000] django-event-tracker indiriliyor... (19 dosya)
   [8/7000] wolverine indiriliyor... (8 dosya)
   [9/7000] epub-creator indiriliyor... (18 dosya)
   [10/7000] django-notices indiriliyor... (6 dosya)
   [11/7000] simplemigrations indiriliyor... (9 dosya)
   [12/7000] django-proxy indiriliyor... (12 dosya)
   [13/7000] django-superflatpages indiriliyor... (8 dosya)
   [14/7000] django-collector indiriliyor... (28 dosya)
   [15/7000] django-courses indiriliyor... (6 dosya)
   [16/7000] baconfile-python indiriliyor... (2 dosya)
   [17/7000] django-metrics indiriliyor... (3 do

In [7]:
df["Kod"][0]

'\n\n# ==========================\n# FILE: bin/calcdeps.py\n# ==========================\n#!/usr/bin/env python\n#\n# Copyright 2006 Google Inc. All Rights Reserved.\n\n\n"""Calculates Javascript dependencies without requiring Google3.\n\nIt iterates over a number of search paths and builds a dependency tree.  With\nthe inputs provided, it walks the dependency tree and outputs all the files\nrequired for compilation.\\n\n"""\n\n\n\n\n\nimport distutils.version\nimport logging\nimport optparse\nimport os\nimport re\nimport subprocess\nimport sys\n\n\n\nreq_regex = re.compile(\'goog\\.require\\s*\\(\\s*[\\\'\\"]([^\\)]+)[\\\'\\"]\\s*\\)\')\nprov_regex = re.compile(\'goog\\.provide\\s*\\(\\s*[\\\'\\"]([^\\)]+)[\\\'\\"]\\s*\\)\')\nns_regex = re.compile(\'^ns:((\\w+\\.)*(\\w+))$\')\nversion_regex = re.compile(\'[\\.0-9]+\')\n\n\ndef IsValidFile(ref):\n  """Returns true if the provided reference is a file and exists."""\n  return os.path.isfile(ref)\n\n\ndef IsJsFile(ref):\n  """Returns true